# DEMAND MD INTRA SPAIN

notebook to generate the file demand_ES_MD_intra.v#.csv
It makes use of the matrices generated by MobA

In [11]:
# libraries to import
import pandas as pd
import numpy as np
from typing import Dict
import os
print(os.getcwd())
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())
pd.set_option('display.max_columns', None)

C:\Users\LMENENDEZ\GitHub\MultiModX
C:\Users\LMENENDEZ\GitHub\MultiModX


In [12]:
# O-D matrices from MobA
trips_for_demand = pd.read_csv(
    r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\Matrices MITMA\with_archetypes\20220922_28_trip_matrix_arc_pt_processed.csv.gz",
    compression="gzip",
    sep="|")

In [13]:
#select trips that happened on friday
trips_for_demand = trips_for_demand[trips_for_demand["date"] == 20220923].reset_index(drop=True).rename(columns={"origin_nut": "origin", "destination_nut": "destination"})

For this notebook I constructed a dictionary with the airports and their associated NUTS because I did not need information about the ports.
In the future, I could use the qgis files form MobA to associate the origine zone and the destination zone directly with the NUTS.

In [14]:
#associates each airport to the corresponding new NUTS
airports_to_NUT={"airport_LPA":("ES705","Gran Canaria"),
                 "airport_FUE":("ES704","Fuerteventura"),
                 "airport_ACE":("ES708","Lanzarote"),
                 "airport_TFS":("ES709","Tenerife"),
                 "airport_TFN":("ES709","Tenerife"),
                 "airport_SPC":("ES707","La Palma"),
                 "airport_VDE":("ES703","El Hierro"),
                 "airport_PMI":("ES532","Mallorca"),
                 "airport_IBZ":("ES531","Eivissa i Formentera"),
                 "airport_MAH":("ES533","Menorca")}

In [16]:
#format trips
trips_for_demand["mode_tp"] = trips_for_demand["mode_sequence"].str.replace("bus", "road") # replace bus to road (some people can reach infrastructure by bus)
trips_for_demand["mode_tp"] = trips_for_demand["mode_tp"].apply(lambda row: [mode for mode in row.split("-") if mode != "road"]) # remove road (it will be considered like access time)
trips_for_demand = trips_for_demand[~trips_for_demand["mode_tp"].apply(lambda x: "ship" in x)] # only considers trips that contain no ship leg
# change aggregated island nuts to dis-aggregate nuts
for key in airports_to_NUT.keys():
    trips_for_demand.loc[trips_for_demand["start_node"]==key,["origin","origin_name"]]=[airports_to_NUT[key][0],airports_to_NUT[key][1]] #change start node
    trips_for_demand.loc[trips_for_demand["end_node"]==key,["destination","destination_name"]]=[airports_to_NUT[key][0],airports_to_NUT[key][1]] #change destination node



In [ ]:
#removes trips from Guadalajara to Madrid and from Madrid to Guadalajara
trips_for_demand=trips_for_demand[~(((trips_for_demand["origin"]=="ES424")&(trips_for_demand["destination"]=="ES300"))|((trips_for_demand["origin"]=="ES300")&(trips_for_demand["destination"]=="ES424")))]

In [18]:
trips_small=trips_for_demand[["date","origin","destination","archetype_0","archetype_1","archetype_2","archetype_3","archetype_4","archetype_5"]]

In [19]:
# aggregates the archetypes
for i in range(0,6):
    globals()[f"df_{i}"]=trips_small.groupby(["date","origin","destination"], as_index = False).agg({f"archetype_{i}":"sum"})

In [20]:
dataframes = [
    (df_0,"archetype_0"),
    (df_1, 'archetype_1'),
    (df_2, 'archetype_2'),
    (df_3, 'archetype_3'),
    (df_4, 'archetype_4'),
    (df_5, 'archetype_5')
]

# Combine the DataFrames with a new "archetype" column
combined_df = pd.concat(
    [
        df.assign(archetype=archetype_col)  # Add new archetype column
        .rename(columns={archetype_col: 'trips'})  # Rename archetype column value to "value"
        for df, archetype_col in dataframes
    ],
    ignore_index=True
)

print(combined_df)


           date  origin destination        trips    archetype
0      20220923   ES111       ES112   874.728102  archetype_0
1      20220923   ES111       ES113  1115.146464  archetype_0
2      20220923   ES111       ES114  4361.562918  archetype_0
3      20220923   ES111       ES120    29.823782  archetype_0
4      20220923   ES111       ES130    12.683604  archetype_0
...         ...     ...         ...          ...          ...
13975  20220923  abroad       ES705     1.161005  archetype_5
13976  20220923  abroad       ES707     0.100615  archetype_5
13977  20220923  abroad       ES708     0.198524  archetype_5
13978  20220923  abroad       ES709     4.614300  archetype_5
13979  20220923  abroad      abroad     0.000000  archetype_5

[13980 rows x 5 columns]


In [21]:
# change column order to match previous version
columns=["date","origin","destination","archetype","trips"]
combined_df=combined_df[columns]

In [22]:
# order the values to match previous version ordering
final_df=combined_df.sort_values(by=["origin","destination","archetype"])

In [23]:
# delete trips that originate abroad and finalise abroad
demand_ES_MD_intra=final_df[~((final_df["origin"]=="abroad")&(final_df["destination"]=="abroad"))]

In [24]:
# delete rows that have 0 trips
demand_ES_MD_intra=demand_ES_MD_intra[~(demand_ES_MD_intra["trips"]==0)]

In [26]:
demand_ES_MD_intra.to_csv("demand_ES_MD_intra_v0.3.csv",index=False)